In [1]:
from matplotlib import pyplot as plt
from matplotlib.image import imread
from os import listdir 
from numpy import asarray
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [2]:
import sys
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [5]:
train_dir = 'train/'
validation_dir='validation/'


epoch = 25
batchsize = 28
train_sample = 5000
validation_sample = 500

train_datagen=ImageDataGenerator(rescale=1.0/255.0,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_it=train_datagen.flow_from_directory(train_dir,class_mode='binary',batch_size=batchsize,target_size=(224,224))
test_it=test_datagen.flow_from_directory(validation_dir,class_mode='binary',batch_size=batchsize,target_size=(224,224))


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [32]:
vgg = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)

AttributeError: 'Model' object has no attribute 'shape'

In [7]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
for layers in vgg.layers:
    layers.trainable = False

    

In [14]:
vgg.output

<tf.Tensor 'block5_pool/Identity:0' shape=(None, 7, 7, 512) dtype=float32>

In [40]:
flat = Flatten()(vgg.output)
dense = Dense(128,activation='relu')(flat)
predict = Dense(1,activation='sigmoid')(dense)
model = Model(inputs=vgg.inputs,outputs=predict)
# model.summary()

In [41]:


# Compiling model with Optimizer and Loss
opt=SGD(learning_rate=0.001,momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])
# model.summary()

In [42]:
history = model.fit_generator(train_it,steps_per_epoch=train_sample // batchsize,
                              validation_data=test_it,validation_steps=validation_sample // batchsize,
                              epochs=epoch,verbose=1)

Epoch 1/25
 45/178 [======>.......................] - ETA: 1:17:43 - loss: 0.4569 - accuracy: 0.7621

KeyboardInterrupt: 